## Hybrid search with Qdrant

Vector search based on dense embeddings captures the semantics of the data, so you don't have to use the same terms in queries and documents to still be able to find the relevant items. However, historically we were also using some other methods which rely on the presence of the keywords. Methods such as Bag-of-Words, TFIDF and BM25 are still useful and in some cases should be preferred over the dense embeddings.

### Sparse vectors

Surprisingly, keyword-based search is also implemented as vector search, but these vectors are usually sparse. That means the majority of the dimensions of such a vector are just zeros. A non-zero value at a particular vector dimension indicates the presence of a term from the dictionary assigned to that position. In other words, in sparse vectors, we have a dictionary in which each word/phrase gets its unique position. Since vectors are sparse, the dictionary can theoretically grow indefinitely, as we can append a new term at the very end.

The fact of using a flexible dictionary, make the sparse vectors excel in exact matches, as they can cover texts that would be sets of random characters for the dense vectors - such as proper names or identifiers. Dense embedding models also have a dictionary, but once the model is trained, extending them is not that easy, and requires fine-tuning of the model. A typical user rarely goes that far.

### BM25

There are plenty of different options for creating sparse embeddings, but BM25 is an industry standard, and its most popular form comes from the 90s. It's a statistical model (no neural networks involved), which makes it really fast and lightweight. It's usually a solid baseline in search benchmarks so you should not ignore it.

BM25 stands for Best Matching 25, and it was just the 25th attempt to create a formula that calculates how relevant a particular document is, given a query. If you are interested in mathematical background, please check out the [Wikipedia page](https://en.wikipedia.org/wiki/Okapi_BM25) that describes it really well. In general, BM25 is a ranking function that helps search engines determine how relevant a document is to a query by combining two key concepts: **Term Frequency (TF)** and **Inverse Document Frequency (IDF)**.

1. The Term Frequency component rewards documents that contain the query terms multiple times, but with diminishing returns - so a document with 10 occurrences of a word isn't necessarily 10 times better than one with just 1 occurrence.
2. The Inverse Document Frequency part boosts the importance of rare words while reducing the weight of common words that appear in many documents, since rare terms are typically more informative for distinguishing relevant results.

BM25 also incorporates document length normalization to prevent longer documents from having an unfair advantage simply due to their size.

In our case, we'll use an implementation available in FastEmbed. Let's start with the basics.

## Step 0: Setup

Please refer to sematic_search.ipynb notebook to set up the libraries required to interact with Qdrant and to create the embeddings. Similarly, please start Qdrant in a Docker container as described there, if it's not running on your machine yet.

If you skipped our previous lessons, the following commands will install all the packages and run Qdrant in a background container.

In [ ]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

In [ ]:
!docker run -d -p 6333:6333 -p 6334:6334 \
   -v "./qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant

In [ ]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost:6333")
client.get_collections()